In [1]:
import sys
import os

# Add the parent directory to sys.path if models.py is in the parent folder
sys.path.append(os.path.abspath('..'))

from models import GatheringModel, read_input, compute_multiphase_pressure_drop
import pandas as pd
from gamspy import Options, Ord

In [2]:
m = GatheringModel("Prototipo", data= read_input("toy_problem.xlsx"))

--- Job _G8MCQ2N4Tzmt4P9o5FxNjA.gms Start 09/04/25 15:36:12 49.6.1 55d34574 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\Diego\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    Input C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpjmuro3z_\_G8MCQ2N4Tzmt4P9o5FxNjA.gms
    Output C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpjmuro3z_\_G8MCQ2N4Tzmt4P9o5FxNjA.lst
    ScrDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpjmuro3z_\tmpm_h0jbjq\
    SysDir C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\
    LogOption 3
    Trace C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpjmuro3z_\_G8MCQ2N4Tzmt4P9o5FxNjA.txt
    License C:\Users\Diego\Documents\GAMSPy\gamspy_license.txt
    OptFile 0
    OptDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpjmuro3z_\
    LimRow 0
    LimCol 0
    TraceOpt 3
    SolPrint 0


In [3]:
model = m.m.models["Multiphase_network_design"]

In [4]:
# summary = model.solve(solver="conopt", options=Options(iteration_limit=2), solver_options={"rtmaxv": "1.e12"})
summary = model.solve(solver="gurobi", options=Options(relative_optimality_gap=0.0001, time_limit = 300.0))

--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(9) 9 Mb
--- Job _G8MCQ2N4Tzmt4P9o5FxNjA.gms Start 09/04/25 15:36:44 49.6.1 55d34574 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\Diego\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    MIQCP gurobi
    Input C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpjmuro3z_\_G8MCQ2N4Tzmt4P9o5FxNjA.gms
    Output C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpjmuro3z_\_G8MCQ2N4Tzmt4P9o5FxNjA.lst
    ScrDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpjmuro3z_\tmpm_h0jbjq\
    SysDir C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\
    LogOption 3
    Trace C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpjmuro3z_\_G8MCQ2N4Tzmt4P9o5FxNjA.txt
    License C:\Users\Diego\Documents\GAMSPy\gamspy_license.txt
    OptFile 0
    OptDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpjmuro3z_\

--- Generating MIQCP model Multiphase_network_design
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(413) 25 Mb
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(413) 26 Mb
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(413) 38 Mb
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(413) 40 Mb
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(420) 51 Mb
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(420) 58 Mb
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(420) 68 Mb
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(420) 70 Mb
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(427) 72 Mb
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(455) 78 Mb
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(511) 82 Mb
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(539) 91 Mb
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(553) 93 Mb
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(581) 101 Mb
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(581) 103 Mb
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(623) 111 Mb
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(623) 114 Mb
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(623) 127 Mb
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(623) 131 Mb
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(623) 148 Mb
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(623) 153 Mb
--- _G8MCQ2N4Tzmt4P9o5FxNjA.gms(679)

[MODEL - WARNING] The solve was interrupted! Solve status: ResourceInterrupt. For further information, see https://gamspy.readthedocs.io/en/latest/reference/gamspy._model.html#gamspy.SolveStatus.


In [5]:
summary

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Resource,Integer,39.256653,82501,378577,MIQCP,GUROBI,305.422


In [6]:
model.toGams(path="prueba")

[CONVERTER - INFO] GAMS (.gms) file has been generated under prueba\Multiphase_network_design.gms


In [13]:
df_var = m.m["x_bar"].records
df_var[df_var["level"] > 0.5]

,n,nn,d,t,level,marginal,lower,upper,scale
162,i1,j2,d1,t15,1.0,NaN,0.0,1.0,1.0
6036,i2,j2,d4,t18,1.0,NaN,0.0,1.0,1.0
11873,i3,j3,d2,t21,1.0,NaN,0.0,1.0,1.0
17266,i4,j3,d1,t23,1.0,NaN,0.0,1.0,1.0
23744,i5,j5,d4,t2,1.0,NaN,0.0,1.0,1.0
29094,i6,j5,d1,t35,1.0,NaN,0.0,1.0,1.0
35483,i7,j7,d1,t36,1.0,NaN,0.0,1.0,1.0
41466,i8,j8,d3,t37,1.0,NaN,0.0,1.0,1.0
47412,i9,j10,d1,t1,1.0,NaN,0.0,1.0,1.0
53221,i10,j10,d2,t13,1.0,NaN,0.0,1.0,1.0


In [14]:
idx = df_var.columns.get_loc("level")

In [17]:
df = df_var.iloc[:, :idx+1]

In [21]:
key_cols = df.columns[:idx]

In [23]:
mask = (df[key_cols] == ('i12', 'j10', 'd1', 't19')).all(axis=1)

In [24]:
row = df.loc[mask, "level"]

In [22]:
key_cols

Index(['n', 'nn', 'd', 't'], dtype='object')

In [11]:
container["y_pf"].

Variable(name='y_pf', domain=[Set(name='pf', domain=[Set(name='n', domain=['*'])]), Set(name='s', domain=['*']), Set(name='t', domain=['*'])], type='binary')